# Phase 2

## Importy

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os 

TARGET = "oximtery"

# naciatanie dat 
DATA_DIR = "./115"

df_obs = pd.read_csv(os.path.join(DATA_DIR, "observation.csv"), sep='\t', encoding='utf-8-sig')
df_pat = pd.read_csv(os.path.join(DATA_DIR, "patient.csv"), sep='\t', encoding='utf-8-sig')
df_sta = pd.read_csv(os.path.join(DATA_DIR, "station.csv"), sep='\t', encoding='utf-8-sig')


## 2.1 Realizácia predspracovaina dát

### Predpriprava

Dame si vyprintovat tabulku observation aby sme mali rychly nahlad do toho s cim pracujeme

In [11]:
def dataset_summary(df, name):
    print("Dataset:",name)
    print("Počet riadkov: ",df.shape[0])
    print("Počet stĺpcov: ",df.shape[1])
    display(df.head())

dataset_summary(df_obs, "observation.csv")

Dataset: observation.csv
Počet riadkov:  12169
Počet stĺpcov:  23


,SpO₂,HR,PI,RR,EtCO₂,FiO₂,PRV,BP,Skin Temperature,Motion/Activity index,...,CO,Blood Flow Index,PPG waveform features,Signal Quality Index,Respiratory effort,O₂ extraction ratio,SNR,oximetry,latitude,longitude
0,96.634591,80.009013,8.685891,17.518665,41.509058,45.163316,144.295418,104.803180,34.978370,10.828257,...,4.088809,40.790753,53.666253,62.281085,54.919133,0.300000,38.410597,1.0,25.67927,-80.31727
1,96.640621,74.589540,13.724679,15.588010,40.198284,50.791918,110.819791,103.974975,35.193140,9.818858,...,4.036563,43.207084,58.219436,45.805412,58.818949,0.213967,39.018173,0.0,-23.79750,-48.59278
2,98.146757,85.019981,7.766128,18.747887,41.964458,63.858679,91.591179,106.573202,35.938901,9.767053,...,4.224814,48.594592,56.163501,45.538311,46.333772,0.205624,30.761858,1.0,-5.41667,38.01667
3,98.391300,73.823193,11.114940,15.905309,42.781757,55.433424,59.373133,103.438804,34.843787,12.695630,...,4.030798,51.561544,42.193516,44.665135,27.899934,0.292025,28.550364,1.0,34.56861,105.89333
4,97.391221,78.702018,12.126423,14.923299,38.946213,52.511153,100.147921,102.652685,36.109030,10.813473,...,4.070682,51.560475,45.447019,63.837901,50.321388,0.242503,38.698971,1.0,51.65333,5.28750


V rýchlosti vidíme že všetky stĺpce sú typu float64, nemáme žiadne chýbajúce hodnoty.

Ďalej si rozdelíme dáta na vstupy a výstupy

In [19]:
Y = df_obs["oximetry"]
X = df_obs.drop(columns=["oximetry"])
X = X.drop(columns=["latitude", "longitude"])

display(Y.head(10))
display(X.head(10))

0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
5    0.0
6    0.0
7    1.0
8    1.0
9    0.0
Name: oximetry, dtype: float64

,SpO₂,HR,PI,RR,EtCO₂,FiO₂,PRV,BP,Skin Temperature,Motion/Activity index,PVI,Hb level,SV,CO,Blood Flow Index,PPG waveform features,Signal Quality Index,Respiratory effort,O₂ extraction ratio,SNR
0,96.634591,80.009013,8.685891,17.518665,41.509058,45.163316,144.295418,104.803180,34.978370,10.828257,14.037362,15.057155,90.483653,4.088809,40.790753,53.666253,62.281085,54.919133,0.300000,38.410597
1,96.640621,74.589540,13.724679,15.588010,40.198284,50.791918,110.819791,103.974975,35.193140,9.818858,14.088573,14.188352,77.917910,4.036563,43.207084,58.219436,45.805412,58.818949,0.213967,39.018173
2,98.146757,85.019981,7.766128,18.747887,41.964458,63.858679,91.591179,106.573202,35.938901,9.767053,15.574370,14.773033,89.239098,4.224814,48.594592,56.163501,45.538311,46.333772,0.205624,30.761858
3,98.391300,73.823193,11.114940,15.905309,42.781757,55.433424,59.373133,103.438804,34.843787,12.695630,13.884048,15.141773,81.060225,4.030798,51.561544,42.193516,44.665135,27.899934,0.292025,28.550364
4,97.391221,78.702018,12.126423,14.923299,38.946213,52.511153,100.147921,102.652685,36.109030,10.813473,14.657350,13.834966,75.759984,4.070682,51.560475,45.447019,63.837901,50.321388,0.242503,38.698971
5,98.370074,86.754404,10.286109,17.971480,37.238983,56.432166,69.915445,101.622527,35.472269,9.986186,15.805165,13.513272,80.743617,4.315589,42.849873,54.511558,64.314948,53.294840,0.237593,20.770371
6,98.713169,77.180663,9.763689,18.186233,36.955089,47.046158,70.942694,109.006225,35.937648,9.291926,13.170684,14.997614,79.732704,4.051470,24.130834,33.733448,47.106062,58.549623,0.220012,39.044142
7,98.980978,85.181119,10.789893,16.435032,42.984438,54.918383,81.807949,103.353147,34.076874,10.466959,15.062680,14.569688,78.293959,4.236177,59.471525,51.456295,56.240107,45.414786,0.221723,22.318890
8,96.433581,76.715451,10.164767,16.451191,42.341025,63.552583,163.976927,108.954107,35.356968,10.615501,14.630771,14.775714,82.915962,4.046473,45.764921,36.326702,34.213963,54.418606,0.274618,33.835461
9,97.867503,71.841157,10.839820,14.017102,39.056205,45.204822,93.668655,100.294404,35.994043,10.060609,15.105803,15.546249,85.481459,4.023458,53.844428,29.902117,61.772263,59.858085,0.246622,25.831583
